In [1]:
import av
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from aging.organization.paths import get_experiment_grouped_depth_files
from aging.organization.dataframes import get_experiment
from aging.plotting import save_factory
from toolz import interleave

In [2]:
random.seed(0)

In [3]:
depth_files = get_experiment_grouped_depth_files()

In [4]:
random.shuffle(depth_files['longtogeny_v2_longtogeny_072023'])
random.shuffle(depth_files['ontogeny_males'])
random.shuffle(depth_files['ontogeny_females'])

In [5]:
list(depth_files)

['ontogeny_females',
 'ontogeny_males',
 'dana_ontogeny_dana_ontogeny',
 'dana_ontogeny_raw_data',
 'longtogeny_males',
 'longtogeny_v2_longtogeny_072023',
 'wheel',
 'dlight',
 'klothos']

In [6]:
save_dict = {}
for k in depth_files:
    save_dict[k] = save_factory('/n/groups/datta/win/tmp/' + k + '_depth_bg')

In [7]:
pixel_list = [
    [75, 300],
    [200, 200],
    # [200, 300],
    [325, 300],
    # [200, 400],
    # [250, 350],
]

In [ ]:
for file in tqdm(interleave(
    [
        depth_files["ontogeny_females"],
        depth_files["ontogeny_males"],
        depth_files["longtogeny_v2_longtogeny_072023"],
    ]
)):
    for px_loc in pixel_list:
        if file.suffix == ".avi":
            px = []
            with av.open(file, "r") as container:
                container.streams.video[0].thread_type = "AUTO"
                for frame in map(lambda x: x.to_ndarray(), container.decode(video=0)):
                    px.append(frame[tuple(px_loc)])
            px = pd.Series(px)
        elif file.suffix == ".dat":
            data = np.memmap(str(file), dtype=np.uint16)
            data = data.reshape((-1, 424, 512))
            px = pd.Series(data[:, px_loc[0], px_loc[1]])
        else:
            continue
        fig = plt.figure()
        plt.plot(px)
        plt.axhline(px.median(), color="C1")
        save_dict[get_experiment(file)](
            fig, file.parent.name + f"{px_loc[0]}_{px_loc[1]}"
        )

        fig = plt.figure()
        plt.plot(px.rolling(601, center=True).median())
        plt.axhline(px.median(), color="C1")
        save_dict[get_experiment(file)](
            fig, file.parent.name + f"_rolling_median_{px_loc[0]}_{px_loc[1]}"
        )
        plt.close("all")

0it [00:00, ?it/s]